## Datasource
- [x] `classification_dataset` from Jung+Ninja
- [x] `Another Website` include: amarintv, , ch3thailand, instagram, mgronline,pantip, pptv, sanook, sudsapda, thairath,       tnn2, workpointnews, babyandkids, dailynews	, khaosod,  new18, postjung, praew, siamdara,thaich8,thansettakij, tvpoolonline, brighttv, facebook, matichon, one31, posttoday, ryt9, siamsport, thainews_prd, think_of_living, twitter
- [ ] `social_listening` include: Facebook, Twitter and Instagram
- [x] `Wiki TH`
- [ ] `Pantip` of Knight (data_lm)

## Stucture Files
```
/raw_data_extraction
│   README.md
└───classification_dataset
│   └───scrapy_dailynews
│   |   │   file111.txt
│   └───scrapy_naewna
│   
└───Another Website
│   └───amarintv
│   |   │   file111.txt
│   └───baanlaesuan
│       │   file111.txt
│   
└───social_listening 
│   │   comment_data.txt
│   │   pantip_post_data.txt
│   
└───Wiki TH
│   └───AA
│       │   file111.txt
│   
└───Pantip 
│   │   file011.txt
```

### Classification_dataset from Jung + Ninja
```
print(data['title'])
print(data['author'])
print(data['publish'])
print(data['content_id'])
print(data['content'])
print(data['url'])
print(data['images'])
print(data['content_class'])
```

In [1]:
import json
import os
import pandas as pd
import re
from enum import Enum
import os.path

In [2]:
# Clear Files
# !rm -rf /workdir/Code/bma_transformer_model/data/raw_data_extraction/*
# --------

In [3]:
ROOT_DIR = "/workdir/Code/bma_transformer_model/data/raw/"
EXPORT_DIR = "/workdir/Code/bma_transformer_model/data/raw_data_extraction/"

SELECTOR_DIR = ["classification_dataset"]
# SELECTOR_DIR = ["another_website"]
# SELECTOR_DIR = ["thwiki-20200601-extracted"]
# SELECTOR_DIR = ["data_lm"]
# SELECTOR_DIR = ["social_listening"]

MINIMUM_FILE_SIZE = 30000000 #size of file 30 MB 
OUTPUT_Filename = ""
extension = ".txt"
count = 1

In [4]:
def write_file_from_json(filename, data):
    f=open(filename,'a')
    f.write(re.sub('[[|!"_.”-]|'']', '', data['title']))
    f.write('\n')
    f.write(re.sub('[[\n\r\t“|!"_.”-]|'']', '', data['content']))

    # New Content
    f.write('\n')
    f.write('\n')
    return f.tell()

def write_file_from_text(filename, data, TYPE_DATA, idx=0):
    f=open(filename, 'a')
    if (TYPE_DATA == "TEXT"):
        f.write(re.sub('[[|!"_.”-]|'']', '', data))
    elif (TYPE_DATA == "WIKI"):
        data = re.sub('<doc.*', '', data)
        data = re.sub('.*doc>', '', data)
        data = re.sub('[[|!"_.”-]|'']', '', data)
        data = re.sub('[[\n\r\t“|!"_.”-]|'']', '', data)
        f.write(data)
    elif (TYPE_DATA == "dataLM"):
        f.write(data['Title'][idx])
        f.write('\n')
        data = re.sub('[[\n\r\t“|!"_.”-]|'']', '', data['Kratoo'][idx])
        f.write(data)
    elif (TYPE_DATA == "social_listening"):
        f.write(data['Title'][idx])
        f.write('\n')
        data = re.sub('[[\n\r\t“|!"_.”-]|'']', '', data['Kratoo'][idx])
        f.write(data)

    # New Content
    f.write('\n')
    f.write('\n')
    return f.tell()

def create_folder(pathFile, count):
    if not os.path.exists(os.path.dirname(pathFile)):
        try:
            os.makedirs(os.path.dirname(pathFile))
            count = 0
            return count
        except OSError as exc: # Guard against race condition
            raise
    return count

def readAndWriteFromJSONFile_LimitFileSize(pathFile, output_file, count):
    # Read JSON File
    with open(pathFile) as json_file:
        data = json.load(json_file)
        
        # Write TxT File
        newSize = write_file_from_json(output_file + "_" + str(count) + ".txt", data)
        # IF size of file MINIMUM_FILE_SIZE 
        if(newSize > MINIMUM_FILE_SIZE):
            count += 1
    return count

def readAndWriteFromTEXTFile_LimitFileSize(pathFile, output_file, count, TYPE_DATA):
    try:
        fRead = open(pathFile, "r")
        data = fRead.read()
    except:
        data = ""

    # Write TxT File on Temp File
    newSize = write_file_from_text(output_file + "_" + str(count) + '.txt', data, TYPE_DATA)

    # IF size of file MINIMUM_FILE_SIZE
    if(newSize > MINIMUM_FILE_SIZE):
        count += 1
    return count


def readAndWriteCsvFile_LimitFileSize(pathFile, output_file, count, dataFrame, TYPE_DATA):
    for i in range(0, dataFrame.shape[0]):
        newSize = write_file_from_text(output_file + "_" + str(count) + '.txt', dataFrame, TYPE_DATA, idx=i)

        # IF size of file MINIMUM_FILE_SIZE
        if(newSize > MINIMUM_FILE_SIZE):
            count += 1
    return count
    

### Processing Time
Export `30GB = 10 Min`

In [5]:
for selector_dir in SELECTOR_DIR:
    old_folder_file = ""
    temp_filename = ""
    for root, dirs, files in os.walk(os.path.join(ROOT_DIR, selector_dir)):
        for file in files:
            folder = os.path.basename(root)

            # Type: "classification_dataset"
            if (selector_dir == "classification_dataset" and folder != "full"):
                # Input
                pathFile = os.path.join(root, file)
                print("pathFile", pathFile)
                break
                filename, file_extension = os.path.splitext(pathFile)
                folder_file = os.path.basename(root)

                # Output
                output_file = os.path.join(EXPORT_DIR, selector_dir, folder_file)
                if(file_extension != '.jpg' or file_extension != '.xml' or file_extension != '.py'):
                    count = create_folder(output_file, count)
                    
                    if(old_folder_file != folder_file):
                        count = 0
                        old_folder_file = folder_file
                        
                    #Read & Write New File
                    count = readAndWriteFromJSONFile_LimitFileSize(pathFile, output_file, count)
                    
            elif (selector_dir == "another_website"):
                # Input
                pathFile = os.path.join(root, file)
                folder_file = pathFile.split("/")[-3]

                # Output
                output_file = os.path.join(EXPORT_DIR, selector_dir, folder_file)

                if(old_folder_file != folder_file):
                    count = 0
                    old_folder_file = folder_file

                count = create_folder(output_file, count)
                count = readAndWriteFromTEXTFile_LimitFileSize(pathFile, output_file, count, "TEXT")

            elif (selector_dir == "thwiki-20200601-extracted"):
                # Input
                pathFile = os.path.join(root, file)
                folder_file = pathFile.split("/")[-2]

                # Output
                output_file = os.path.join(EXPORT_DIR, selector_dir, "Wiki"+folder_file)
                
                if(old_folder_file != folder_file):
                    count = 0
                    old_folder_file = folder_file

                count = create_folder(output_file, count)
                count = readAndWriteFromTEXTFile_LimitFileSize(pathFile, output_file, count, "WIKI")
            elif (selector_dir == "data_lm"):
                # Input
                pathFile = os.path.join(root, file)
                folder_file = pathFile.split("/")[-1]

                # Output
                output_file = os.path.join(EXPORT_DIR, selector_dir, "Pantip"+folder_file)
                
                if(old_folder_file != folder_file):
                    count = 0
                    old_folder_file = folder_file
                count = create_folder(output_file, count)
                dataFrame = pd.read_csv(pathFile, delimiter=',', encoding='utf-8')
                count = readAndWriteCsvFile_LimitFileSize(pathFile, output_file, count, dataFrame, "dataLM")
            elif (selector_dir == "social_listening"):
                pathFile = os.path.join(root, file)
                print(">>", pathFile)
                folder_file = pathFile.split("/")[-1]

                # Output
                output_file = os.path.join(EXPORT_DIR, selector_dir, "SocialListening"+folder_file)
                
                if(old_folder_file != folder_file):
                    count = 0
                    old_folder_file = folder_file
                count = create_folder(output_file, count)
                pathFile = "/workdir/Code/bma_transformer_model/data/raw/social_listening/pantip_post_data.csv"
                dataFrame = pd.read_csv(pathFile, encoding='utf-8', delimiter="\t", error_bad_lines=False, names=["A", "B", "C", "D", "E", "F"])
                print(dataFrame["D"])
                

#                 count = readAndWriteCsvFile_LimitFileSize(pathFile, output_file, TEMP_Filename, count, dataFrame, "social_listening")
#             break
#         break
#     break
                

pathFile /workdir/Code/bma_transformer_model/data/raw/classification_dataset/scrapy_pptv36/pptv36/pptv36_55520.json
pathFile /workdir/Code/bma_transformer_model/data/raw/classification_dataset/scrapy_thaipbs/thaipbs/thaipbs_108355.json
pathFile /workdir/Code/bma_transformer_model/data/raw/classification_dataset/scrapy_springnews/springnews/springnews_542304.json
pathFile /workdir/Code/bma_transformer_model/data/raw/classification_dataset/scrapy_siamrath/siamrath/siamrath_156220.json
pathFile /workdir/Code/bma_transformer_model/data/raw/classification_dataset/scrapy_dailynews/dailynews/dailynews_774189.json
pathFile /workdir/Code/bma_transformer_model/data/raw/classification_dataset/scrapy_naewna/naewna/naewna_485187.json
pathFile /workdir/Code/bma_transformer_model/data/raw/classification_dataset/scrapy_prbangkok/prbangkok/prbangkok_MDY1cDBzNnM0NHIyb3Ezc3E2NnEyNDk0cDRyOTQzcjQ1NzIz.json
pathFile /workdir/Code/bma_transformer_model/data/raw/classification_dataset/scrapy_prachachat/pracha